# **Index**  
[0. Preparation](#0-preparation)  
[1. YZiCS Galaxy Load](#1-yzics-galaxy-load)  
[2. Merger Tree Load](#2-merger-tree-load)  
[3. Merger Tree Modify](#3-merger-tree-modify)  
[4. Make Catalogue](#4-make-catalogue)  
[5. Build Model](#5-build-model)  
[6. Train Model](#6-train-model)  
[7. Evaluation](#7-evaluation)  
refer `youngtree_save.py`

# 0. Preparation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
if not "/home/jeon/jeonpkg" in sys.path:
    sys.path.append("/home/jeon/jeonpkg")
    print("/home/jeon/jeonpkg update")
from importlib import reload
import jeon as jn
import jfiles as jf
import jplots as jp
from rur import uri, uhmi, painter, drawer

import os
# from rur import uri, uhmi, painter
import time
from matplotlib.ticker import MultipleLocator
import gc
from collections.abc import Iterable
import inspect
import psutil
gc.collect()
import logging
import copy

import traceback


Module JEON(220503.0) loading success
Module JFIELS(210126.0) loading success
Module JEON(220503.0) loading success
Module JFIELS(210126.0) loading success
Module JPLOTS(220503.0) loading success


In [2]:
from tree_utool import *
import json

In [82]:
ptree = jn.pklload("/storage6/NewHorizon/ptree/ptree_stable.pkl")

In [83]:
ptree.dtype

dtype([('nparts', '<i4'), ('id', '<i4'), ('timestep', '<i4'), ('level', '<i4'), ('host', '<i4'), ('hostsub', '<i4'), ('nbsub', '<i4'), ('nextsub', '<i4'), ('aexp', '<f8'), ('m', '<f8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<f8'), ('Lx', '<f8'), ('Ly', '<f8'), ('Lz', '<f8'), ('r', '<f8'), ('a', '<f8'), ('b', '<f8'), ('c', '<f8'), ('ek', '<f8'), ('ep', '<f8'), ('et', '<f8'), ('spin', '<f8'), ('sigma', '<f8'), ('sigma_bulge', '<f8'), ('m_bulge', '<f8'), ('rvir', '<f8'), ('mvir', '<f8'), ('tvir', '<f8'), ('cvel', '<f8'), ('rho0', '<f8'), ('rc', '<f8'), ('desc', '<i4', (3, 4)), ('npass', '<i4', (3, 4)), ('hmid', '<i4'), ('fat', '<i4'), ('son', '<i4'), ('score_fat', '<f8'), ('score_son', '<f8'), ('nprog', '<i4'), ('ndesc', '<i4'), ('first', '<i4'), ('last', '<i4'), ('first_rev', '<i4'), ('last_rev', '<i4')])

In [92]:
lastids = ptree[(ptree['timestep']==1026)&(ptree['m']>1e8)]['last']

for lastid in lastids:
    temp = ptree[ptree['last']==lastid]
    leng = len(temp)
    if leng<500:
        print(lastid, leng, np.max(np.log10(temp['m'])), temp[temp['timestep']==1026][0]['hmid'])
    # time.sleep(3)

587043 85 9.197125107522496 57
630099 113 9.048605293640504 96
793403 164 8.44775480035543 233
706428 349 9.384983606440073 162
845924 374 8.067311915744204 274
876123 484 9.447127407176781 297
914954 387 8.907513883370106 326
1023569 255 8.61031158454524 404
1182464 463 9.085045031390788 511
1080158 75 8.587761366903539 443
1334768 484 8.7920323591348 607
1329869 304 10.662881554502906 604
1370969 113 9.90754797486633 629
1473876 236 9.681042709456891 690
1525851 73 8.264132117975796 720
1628888 309 8.43818729978173 778
967109 432 10.797336916255878 364
1889513 470 8.556027377994267 917
1897299 11 8.818596506360642 921
2167376 252 9.493302543421859 1055
1993976 198 8.527718995205092 970
1995974 121 9.302819717970209 971
2030093 404 8.974269100125706 988
2052324 424 9.093262278687797 999
1659726 277 8.090862778510127 795
2034126 497 10.281567002474542 990
2134184 349 8.687837668228989 1039
2163213 459 8.111025603494385 1053
11874474 416 8.845637690421201 3846
2165294 268 9.384899444434

In [50]:
a = np.random.randint(1, 1000000, 500000)
b = np.random.randint(1, 1000000, 500000)

In [81]:
leng = 5

b = np.random.randint(1, 1000000, 50000)
aa = [np.random.randint(1, 1000000, 50000) for _ in range(leng)]
clock = timer()
result = np.zeros(leng).astype(bool)
for i in range(leng):
    result[i] = atleast_numba(aa[i],b)
clock.done()
print(result[:10])


clock = timer()
result = atleast_numba_para(aa,b)
clock.done()
print(result[:10])

 START
 Done (0.004 sec)
[ True  True  True  True  True]
 START
 Done (0.037 sec)
[ True  True  True  True  True]


In [38]:
print()

1, 2, 3, ..., 11, 12


In [3]:
jn.stop()

AttributeError: module 'jeon' has no attribute 'stop'

# YOUNGTREE  
Log file남기기?  
progenitor 판단이 잘 되고 있는지 모르겠음 score나 각 value라도 저장하는게 나을듯  

In [10]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

class timer():
    def __init__(self, unit="sec",text="", verbose=2, debugger=None):
        self.ref = time.time()
        self.units = {"ms":1/1000, "sec":1, "min":60, "hr":3600}
        self.corr = self.units[unit]
        self.unit = unit
        self.text = text
        self.verbose=verbose
        self.debugger=debugger
        
        if self.verbose>0:
            print(f"{text} START")
        if self.debugger is not None:
            self.debugger.info(f"{text} START")
    
    def done(self):
        elapse = time.time()-self.ref
        if self.verbose>0:
            print(f"{self.text} Done ({elapse/self.corr:.3f} {self.unit})")
        if self.debugger is not None:
            self.debugger.info(f"{self.text} Done ({elapse/self.corr:.3f} {self.unit})")

def dprint(msg, debugger):
    debugger.debug(msg)
def MB():
    return psutil.Process().memory_info().rss / 2 ** 20
def GB():
    return psutil.Process().memory_info().rss / 2 ** 30

In [11]:
import resource

In [84]:
def load_nout(repo, galaxy=True):
    if galaxy:
        path = f"{repo}/galaxy"
    else:
        path = f"{repo}/halo"

    fnames = np.array(os.listdir(path))
    ind = [True if "tree_bricks" in file else False for file in fnames]
    fnames = fnames[ind]
    fnames = -np.sort( -np.array([int(file[11:]) for file in fnames]) )
    return fnames

def load_nstep(repo, galaxy=True, nout=None):
    if nout is None:
        nout = load_nout(repo, galaxy=galaxy)
    nstep = np.arange(len(nout))[::-1]+1
    return nstep

def out2step(iout, repo, galaxy=True, mode='hagn', nout=None, nstep=None):
    if nout is None:
        nout = load_nout(repo, galaxy=galaxy)
    if nstep is None:
        nstep = load_nstep(repo, galaxy=galaxy, nout=nout)
    arg = np.argwhere(iout==nout)[0][0]
    print( np.argwhere(iout==nout) )
    print( np.where(iout==nout) )
    return nstep[arg]

In [92]:
aa = {1:10, 2:20, 3:30}
aa.items()
b = [f"At {a}: {b}" for a,b in aa.items()]
c = " | ".join(b)
c

'At 1: 10 | At 2: 20 | At 3: 30'

In [ ]:
import textwrap
def wrap(string, max_width):
    return '\n'.join(textwrap.wrap(string,max_width))

In [85]:
repo = "/storage3/Clusters/07206"
nout = load_nout(repo, galaxy=True)
nstep = load_nstep(repo, galaxy=True, nout=nout)
out2step(178, repo, mode='y07206', nout=nout)

[[9]]
(array([9]),)


169

In [75]:
print( nout[:10] )
print( nstep[:10] )


[187 186 185 184 183 182 181 180 179 178]
[178 177 176 175 174 173 172 171 170 169]


# =Class: Treebase=

In [13]:
#########################################################
###############         Tree Class                #######
#########################################################
class Treebase():
    def __init__(self, simmode='hagn', flush_GB=50, verbose=2, debugger=None, loadall=False, prefix=""):
        func = f"[__Treebase__]"; prefix = f"{prefix}{func}"
        clock = timer(text=prefix, verbose=verbose, debugger=debugger)

        self.iniGB = GB()
        self.iniMB = MB()
        self.flush_GB = flush_GB
        self.simmode = simmode
        self.verbose = verbose
        self.debugger = debugger
        if simmode[0] == 'h':
            self.rurmode = 'hagn'
            self.repo = f"/storage4/Horizon_AGN/"
        elif simmode[0] == 'y':
            self.rurmode = 'yzics'
            self.repo = f"/storage3/Clusters/{simmode[1:]}"
        self.loadall = loadall
        self.nout, self.nstep, self.zred, self.aexp, self.gyr = jn.pklload(f"/storage6/jeon/data/{self.simmode}/{self.simmode}_nout_nstep_zred_aexp_gyr.pickle")
        self.dm_nout, self.dm_nstep, self.dm_zred, self.dm_aexp, self.dm_gyr = jn.pklload(f"/storage6/jeon/data/{self.simmode}/dm_{self.simmode}_nout_nstep_zred_aexp_gyr.pickle")
        self.dict_snap = {} # in {iout}, RamsesSnapshot object
        self.dict_star = {} # in {iout}, in {galid}, Particle object
        self.dict_dm = {} # in {iout}, in {haloid}, Particle object
        self.dict_cell = {} # not used
        self.dict_gals = {"galaxymakers":{}, "gmpids":{}} # in each key, in {iout}, Recarray obejct
        self.dict_halos = {"halomakers":{}, "hmpids":{}} # in each key, in {iout}, Recarray obejct
        tm = jn.pklload(f"/storage6/jeon/data/{self.simmode}/tm/{self.simmode}_TreeMaker.pickle")
        self.galtm = jn.treemaker(galaxy=True, tm=tm, mode=self.simmode)
        tm = jn.pklload(f"/storage6/jeon/data/{self.simmode}/tm/{self.simmode}_dm_TreeMaker.pickle")
        self.halotm = jn.treemaker(galaxy=False, tm=tm, mode=self.simmode)
        del tm
        self.saved_out = {"snap":[], "gal":[], "halo":[], "star":[], "dm":[], "cell":[]}
        self.running_mp = False
        self.branches_queue = {}
        gc.collect()

        clock.done()

    def summary(self, isprint=False):
        text = "\n[Tree Data Report]\n\n>>> Snapshot\n"
        a = [f"{key} | " for key in self.dict_snap.keys()]
        for ia in a:
            text+= ia
        text+= "\n>>> Star\n"
        for key in self.dict_star.keys():
            keys = list(self.dict_star[key].keys())
            text += f"\t{key}: {len(self.dict_star[key])} gals with {np.sum([len(self.dict_star[key][ia]['id']) for ia in keys])} stars\n"
            for ikey in keys:
                text += f"\t\tID{ikey:05d} Nparts{len(self.dict_star[key][ikey]['id'])}\n"
        text+= ">>> DM\n"
        for key in self.dict_dm.keys():
            keys = list(self.dict_dm[key].keys())
            text += f"\t{key}: {len(self.dict_dm[key])} halos with {np.sum([len(self.dict_dm[key][ia]['id']) for ia in keys])} DMs\n"
            for ikey in keys:
                text += f"\t\tID{ikey:05d} Nparts{len(self.dict_dm[key][ikey]['id'])}\n"
        text+= ">>> GalaxyMaker\n"
        for key in self.dict_gals["galaxymakers"].keys():
            text += f"\t{key}: {len(self.dict_gals['galaxymakers'][key])} gals with {np.sum([len(ia) for ia in self.dict_gals['gmpids'][key]])} stars\n"
        text+= ">>> HaloMaker\n"
        for key in self.dict_halos["halomakers"].keys():
            text += f"\t{key}: {len(self.dict_halos['halomakers'][key])} halos with {len(self.dict_halos['hmpids'][key])} DMs\n"
        text+= f">>> Used Memory: {psutil.Process().memory_info().rss / 2 ** 30:.4f} GB\n"
        
        if isprint:
            print(text)
        return text
        
        
    def make_branches(self, iout, galids=None, galaxy=True, prefix=""):
        # Subject to queue
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

        if galaxy:
            gals = self.load_gal(iout, galid=galids, prefix=prefix)
        else:
            gals = self.load_halo(iout, haloid=galids, prefix=prefix)

        for gal in gals:
            self.branches_queue[gal['id']]=Branch(gal, self, galaxy=galaxy, mode=self.simmode, verbose=self.verbose, prefix=prefix, debugger=self.debugger,trust_tree=False)

        clock.done()

    def queue(self, iout, gals, treeleng=None, galaxy=True, prefix=""):
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

        self.make_branches(iout, gals['id'], galaxy=galaxy, prefix=prefix)
        self.debugger.info(f"{prefix}\n{self.summary()}")
        try:
            go=True
            ntree = 1
            while go:
                keys = list( self.branches_queue.keys() )
                for key in keys:
                    go = False
                    iprefix = prefix+f"<{key:05d}> "
                    igo = self.branches_queue[key].do_onestep(prefix=iprefix)
                    igo = igo & self.branches_queue[key].go
                    if not igo:
                        self.branches_queue[key].selfsave()
                        self.branches_queue[key] = None
                        del self.branches_queue[key]
                    else: go=True
                    gc.collect()
                self.flush_auto(prefix=prefix)
                self.debugger.info(f"{prefix}\n{self.summary()}")
                ntree += 1
                if treeleng is not None:
                    if ntree >= treeleng:
                        go=False
        except Exception as e:
            print(traceback.format_exc())
            print(e)
            self.debugger.error(traceback.format_exc())
            self.debugger.error(e)
            self.debugger.error(self.summary())

        clock.done()

    # def prepare(self):
    #     # 1. Check current branches
    #     # 2. Prepare needed snaps
    #     # 3. Prepare needed galaxies(halos)
    #     # 4. Prepare needed stars(dms)
    #     func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
    #     clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)
        
    #     minout = np.inf; maxout = 0
    #     for branch in self.branches:
    #         cands_dict = branch.candidates
    #         iouts = cands_dict.keys()
    #         minout = min(minout, np.min(iouts))
    #         maxout = max(maxout, np.max(iouts))
    #     minstep = jn.out2step(minstep, galaxy=branch.galaxy, mode=self.simmode)
    #     if minstep > np.min(self.nstep):
    #         for i in range(4):
    #             if minstep-i < np.min(self.nstep):
    #                 break
    #             iiout = jn.step2out(minstep-i, galaxy=branch.galaxy, mode=self.simmode)
    #             self.load_snap(self, iiout, prefix=prefix)


    #     pass

    def check_outs(self, prefix=""):
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        # clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

        self.saved_out["snap"] = list(self.dict_snap.keys())
        self.saved_out["gal"] = list(self.dict_gals["galaxymakers"].keys())
        self.saved_out["halo"] = list(self.dict_halos["halomakers"].keys())
        self.saved_out["star"] = list(self.dict_star.keys())
        self.saved_out["dm"] = list(self.dict_dm.keys())
        self.saved_out["cell"] = list(self.dict_cell.keys())
        # clock.done()

    def flush_auto(self, prefix=""):
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)
        reftot = MB()

        cout = -1
        keys = list( self.dict_snap.keys() )
        if len(keys)>0:
            cout = max(np.min(keys), cout)
        keys = list( self.dict_gals["galaxymakers"].keys() )
        if len(keys)>0:
            cout = max(np.min(keys), cout)
        keys = list( self.dict_halos["halomakers"].keys() )
        if len(keys)>0:
            cout = max(np.min(keys), cout)
        keys = list( self.dict_star.keys() )
        if len(keys)>0:
            cout = max(np.min(keys), cout)
        keys = list( self.dict_dm.keys() )
        if len(keys)>0:
            cout = max(np.min(keys), cout)
        keys = list( self.dict_cell.keys() )
        if len(keys)>0:
            cout = max(np.min(keys), cout)
        
        # Snapshot
        keys = list( self.dict_snap.keys() )
        if len(keys)>0:
            for iout in keys:
                if iout > cout+4:
                    refmem = MB()
                    self.dict_snap[iout].clear()
                    self.dict_snap[iout] = None
                    del self.dict_snap[iout]
                    gc.collect()
                    self.debugger.info(f"* [flush][snapshot] remove iout={iout} ({refmem-MB():.2f} MB saved)")
        # GalaxyMaker
        keys = list( self.dict_gals["galaxymakers"].keys() )
        if len(keys)>0:
            for iout in keys:
                if iout > cout+4:
                    refmem = MB()
                    self.dict_gals['galaxymakers'][iout] = None
                    self.dict_gals['gmpids'][iout] = None
                    del self.dict_gals['galaxymakers'][iout]
                    del self.dict_gals['gmpids'][iout]
                    gc.collect()
                    self.debugger.info(f"* [flush][GalaxyMaker] remove iout={iout} ({refmem-MB():.2f} MB saved)")
        # HaloMaker
        keys = list( self.dict_halos["halomakers"].keys() )
        if len(keys)>0:
            for iout in keys:
                if iout > cout+4:
                    refmem = MB()
                    self.dict_halos['halomakers'][iout] = None
                    self.dict_halos['hmpids'][iout] = None
                    del self.dict_halos['halomakers'][iout]
                    del self.dict_halos['hmpids'][iout]
                    gc.collect()
                    self.debugger.info(f"* [flush][HaloMaker] remove iout={iout} ({refmem-MB():.2f} MB saved)")
        # Star
        keys = list( self.dict_star.keys() )
        if len(keys)>0:
            for iout in keys:
                if iout > cout+4:
                    jkeys = list(self.dict_star[iout].keys())
                    for galid in jkeys:
                        refmem = MB()
                        self.dict_star[iout][galid].table = []
                        self.dict_star[iout][galid].snap.clear()
                        self.dict_star[iout][galid] = None
                        del self.dict_star[iout][galid]
                        gc.collect()
                        self.debugger.info(f"* [flush][Star] remove iout={iout} gal={galid} ({refmem-MB():.2f} MB saved)")
                    refmem = MB()
                    self.dict_star[iout] = None
                    del self.dict_star[iout]
                    gc.collect()
                    self.debugger.info(f"* [flush][Star] remove iout={iout} garbage ({refmem-MB():.2f} MB saved)")
        # DM
        keys = list( self.dict_dm.keys() )
        if len(keys)>0:
            for iout in keys:
                if iout > cout+4:
                    jkeys = list(self.dict_dm[iout].keys())
                    for haloid in jkeys:
                        refmem = MB()
                        self.dict_dm[iout][haloid].table = []
                        self.dict_dm[iout][haloid].snap.clear()
                        self.dict_dm[iout][haloid] = None
                        del self.dict_dm[iout][haloid]
                        gc.collect()
                        self.debugger.info(f"* [flush][DM] remove iout={iout} halo={haloid} ({refmem-MB():.2f} MB saved)")
                    refmem = MB()
                    self.dict_dm[iout] = None
                    del self.dict_dm[iout]
                    gc.collect()
                    self.debugger.info(f"* [flush][DM] remove iout={iout} garbage ({refmem-MB():.2f} MB saved)")
        # Cell
        keys = list( self.dict_cell.keys() )
        if len(keys)>0:
            for iout in keys:
                if iout > cout+4:
                    self.dict_cell[iout] = None
                    del self.dict_cell[iout]
                    gc.collect()
        
        self.check_outs(prefix=prefix)
        self.debugger.info(f"* [flush][Total] {reftot-MB():.2f} MB saved")

        clock.done()

    def load_snap(self, iout, prefix=""):
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        # clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)
        if psutil.Process().memory_info().rss / 2 ** 30 > self.flush_GB+self.iniGB:
            self.flush_auto(prefix=prefix)

        if not iout in self.dict_snap.keys():
            self.dict_snap[iout] = uri.RamsesSnapshot(self.repo, iout, mode=self.rurmode, path_in_repo='snapshots')
        if not iout in self.dict_star.keys():
            self.dict_star[iout] = {}
        if not iout in self.dict_dm.keys():
            self.dict_dm[iout] = {}
        if not iout in self.dict_cell.keys():
            self.dict_cell[iout] = {}
        self.check_outs(prefix=prefix)

        # clock.done()
        return self.dict_snap[iout]

    def load_gal(self, iout, galid, return_part=False, prefix="", fromtm=False):
        # func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        # clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)
        if psutil.Process().memory_info().rss / 2 ** 30 > self.flush_GB+self.iniGB:
            self.flush_auto(prefix=prefix)

        if fromtm:
            # clock.done()
            return self.galtm.loadgals(iout, galid)
        else:
            if not iout in self.dict_gals["galaxymakers"].keys():
                func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
                # clock2 = timer(text=prefix+"[GalaxyMaker load]", verbose=self.verbose, debugger=self.debugger)
                snap = self.load_snap(iout, prefix=prefix)
                gm, gmpid = uhmi.HaloMaker.load(snap, galaxy=True, load_parts=True)
                self.dict_gals["galaxymakers"][iout] = copy.deepcopy(gm)
                cumparts = np.insert(np.cumsum(gm["nparts"]), 0, 0)
                gmpid = [gmpid[ cumparts[i]:cumparts[i+1] ] for i in range(len(gm))]
                self.dict_gals["gmpids"][iout] = copy.deepcopy(gmpid)
                del gm; del gmpid; del cumparts
                gc.collect()
                if self.loadall:
                    self.debugger.info(f"{prefix} *** loadall=True: loadall stars...")
                    snap.box = np.array([[0, 1], [0, 1], [0, 1]])
                    snap.get_part()
                    for gal in self.dict_gals["galaxymakers"][iout]:
                        self.load_star(iout, gal['id'], prefix=prefix, silent=True)
                    self.debugger.info(f"{prefix} *** loadall=True: loadall stars Done")
                self.check_outs(prefix=prefix)
                # clock2.done()

            # clock.done()
            if isinstance(galid,str):
                if galid=='all':
                    return self.dict_gals["galaxymakers"][iout]
            elif isinstance(galid, Iterable):
                a = np.hstack([self.dict_gals["galaxymakers"][iout][ia-1] for ia in galid])
                if return_part:
                    b = [self.dict_gals["gmpids"][iout][ia-1] for ia in galid]
                    return a, b
                return a
            else:
                if return_part:
                    return self.dict_gals["galaxymakers"][iout][galid-1], self.dict_gals["gmpids"][iout][galid-1]
                return self.dict_gals["galaxymakers"][iout][galid-1]
    
    def load_halo(self, iout, haloid, return_part=False, prefix="", fromtm=False):
        # func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        # clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)
        if psutil.Process().memory_info().rss / 2 ** 30 > self.flush_GB+self.iniGB:
            self.flush_auto(prefix=prefix)

        if fromtm:
            # clock.done()
            return self.galtm.loadgals(iout, haloid)
        else:
            if not iout in self.dict_halos["halomakers"].keys():
                func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
                # clock2 = timer(text=prefix+"[HaloMaker load]", verbose=self.verbose, debugger=self.debugger)
                snap = self.load_snap(iout, prefix=prefix)
                gm, gmpid = uhmi.HaloMaker.load(snap, galaxy=False, load_parts=True)
                self.dict_halos["halomakers"][iout] = copy.deepcopy(gm)
                cumparts = np.insert(np.cumsum(gm["nparts"]), 0, 0)
                gmpid = [gmpid[ cumparts[i]:cumparts[i+1] ] for i in range(len(gm))]
                self.dict_halos["hmpids"][iout] = copy.deepcopy(gmpid)
                del gm; del gmpid; del cumparts
                gc.collect()
                if self.loadall:
                    self.debugger.info(f"{prefix} *** loadall=True: loadall DMs...")
                    snap.box = np.array([[0, 1], [0, 1], [0, 1]])
                    snap.get_part()
                    for halo in self.dict_halos["halomakers"][iout]:
                        self.load_dm(iout, halo['id'], prefix=prefix, silent=True)
                    self.debugger.info(f"{prefix} *** loadall=True: loadall DMs Done")
                self.check_outs(prefix=prefix)
                # clock2.done()

            # clock.done()
            if haloid == 'all':
                return self.dict_halos["halomakers"][iout]
            if return_part:
                return self.dict_halos["halomakers"][iout][haloid-1], self.dict_halos["hmpids"][iout][haloid-1]
            return self.dict_halos["halomakers"][iout][haloid-1]

    def load_star(self, iout, galid, prefix="", silent=False):
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        # clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)
        if psutil.Process().memory_info().rss / 2 ** 30 > self.flush_GB+self.iniGB:
            self.flush_auto(prefix=prefix)

        snap = self.load_snap(iout, prefix=prefix)
        gal, gpid = self.load_gal(iout, galid, return_part=True, prefix=prefix)
        if not galid in self.dict_star[iout].keys():
            if snap.part_data is not None:
                if snap.part_data.nbytes / 2**30 > self.flush_GB:
                    self.debugger.info(prefix+f" snap.clear!! ({snap.part_data.nbytes / 2**30:.2f} GB)")
                    snap.clear()
            leng = 0
            scale = 1
            while leng < len(gpid):
                snap.set_box_halo(gal, 1.1*scale, use_halo_radius=True, radius_name='r')
                if not silent:
                    clock2 = timer(text=prefix+"[get_part]", verbose=self.verbose, debugger=self.debugger)
                snap.get_part()
                if not silent:
                    clock2.done()
                try:
                    star = copy.deepcopy(snap.part['star'].table)
                    star['id'] = np.abs(star['id'])
                    if jn.atleast_isin(star['id'], gpid):
                        star = star[jn.large_isin(star['id'], gpid)]
                        leng = len(star['id'])
                    else:
                        leng = 0
                except: # didin't found star??
                    leng = 0
                scale *= 2
                if scale > 130:
                    break
            star = uri.RamsesSnapshot.Particle(star, snap)
            self.dict_star[iout][galid] = star
            self.check_outs(prefix=prefix)
        
        # clock.done()
        return self.dict_star[iout][galid]

    def load_dm(self, iout, haloid, prefix="", silent=False):
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)
        if psutil.Process().memory_info().rss / 2 ** 30 > self.flush_GB+self.iniGB:
            self.flush_auto(prefix=prefix)

        snap = self.load_snap(iout, prefix=prefix)
        halo, hpid = self.load_halo(iout, haloid, return_part=True)
        if not haloid in self.dict_dm[iout].keys():
            if snap.part_data is not None:
                if snap.part_data.nbytes / 2**30 > self.flush_GB:
                    self.debugger.info(prefix+f" snap.clear!! ({snap.part_data.nbytes / 2**30:.2f} GB)")
                    snap.clear()
            ################## not yet
            snap.set_box_halo(halo, 1.1, use_halo_radius=True, radius_name='r')
            if not silent:
                clock2 = timer(text=prefix+"[get_part]", verbose=self.verbose, debugger=self.debugger)
            snap.get_part()
            if not silent:
                clock2.done()
            dm = copy.deepcopy(snap.part['dm'])
            dm['id'] = np.abs(dm['id'])
            dm = dm[jn.large_isin(dm['id'], hpid)]
            dm = uri.RamsesSnapshot.Particle(dm, snap)
            self.dict_dm[iout][haloid] = dm
            self.check_outs(prefix=prefix)

        clock.done()
        return self.dict_dm[iout][haloid]

# =Class: Leaf=

In [14]:
#########################################################
###############         Leaf Class                #######
#########################################################
class Leaf():
    def __init__(self, gal, BranchObj, DataObj, verbose=1, prefix="", debugger=None, **kwargs):
        func = f"[__Leaf__]"; prefix = f"{prefix}{func}"
        # clock = timer(text=prefix, verbose=verbose, debugger=debugger)
        self.debugger=debugger
        self.verbose = verbose

        self.branch = BranchObj
        self.data = DataObj
        self.mode, self.galaxy = self.branch.mode, self.branch.galaxy
        self.gal_gm = self.branch.tm2gm(gal, prefix=prefix)
        self.gal_tm = self.branch.gm2tm(gal, prefix=prefix)
        self.galid, self.iout, self.istep = self.gal_gm['id'], self.gal_gm['timestep'], self.gal_tm['timestep']
        
        self.havefats = False
        self.refer = 6
        self.part = None
        prefix += f"<ID{self.galid}:iout(istep)={self.iout}({self.istep})>"
        self.load_parts(prefix=prefix)
        self.importance(prefix=prefix, usevel=True)

        # clock.done()

    def clear(self):
        self.part = []

    def load_parts(self, prefix=""):
        # Subject to `__init__`
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

        if self.galaxy:
            self.part = self.data.load_star(self.iout, self.galid, prefix=prefix)
        else:
            self.part = self.data.load_dm(self.iout, self.galid, prefix=prefix)
        self.debugger.debug(prefix+f" [ID{self.galid} iout{self.iout}] Nparts={self.gal_gm['nparts']}({len(self.part['x'])})")

        clock.done()
    

    def importance(self, prefix="", usevel=True):
        # Subject to `__init__`
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        # clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)
    
        cx, cy, cz = self.gal_gm['x'],self.gal_gm['y'],self.gal_gm['z']
        dist = jn.distance3d(cx,cy,cz, self.part['x'], self.part['y'], self.part['z']) / self.gal_gm['rvir']
        if usevel:
            try:
                cvx, cvy, cvz = self.gal_gm['vx'],self.gal_gm['vy'],self.gal_gm['vz']
                vels = jn.distance3d(cvx,cvy,cvz, self.part['vx'], self.part['vy'], self.part['vz'])
                vels /= np.std(vels)
                dist = np.sqrt( dist**2 + vels**2 )
            except Warning as e:
                print("WARNING!! in importance")
                self.debugger.warning("########## WARNING #########")
                self.debugger.warning(e)
                self.debugger.warning(f"gal velocity {cvx}, {cvy}, {cvz}")
                self.debugger.warning(f"len parts {len(self.part['x'])}")
                self.debugger.warning(f"vels first tens {vels[:10]}")
                self.debugger.warning(f"vels std {np.std(vels)}")
                self.debugger.warning(self.summary())
                breakpoint()
                raise ValueError("velocity wrong!")
        table = jn.addcolumn(self.part.table, "dist", dist, dtype='<f8')
        self.part = uri.RamsesSnapshot.Particle(table, self.part.snap)

        # clock.done()
    

    def load_fatids(self, igals, njump=0, masscut_percent=1, nfat=5, prefix="", trust_tree=False, **kwargs):
        # Subject to `find_candidates`
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

        if trust_tree:
            if len(igals) == 1:
                igals = np.concatenate((igals, igals))
            fats = np.vstack((igals['fat1'], igals['fat2'], igals['fat3'], igals['fat4'], igals['fat5']))[:nfat].ravel() # (5, ngal)
            if masscut_percent>0:
                mfats = np.vstack((igals['mfat1'], igals['mfat2'], igals['mfat3'], igals['mfat4'], igals['mfat5']))[:nfat].ravel()
                fats = fats[mfats > masscut_percent]
        else:
            iout, istep = jn.ioutistep(igals[0], galaxy=self.galaxy, mode=self.mode)
            jout = jn.step2out(istep-1-njump, galaxy=self.galaxy, mode=self.mode)
            agals = self.data.load_gal(jout, 'all', return_part=False, prefix=prefix, fromtm=False)
            dt = jn.timeconversion(iout, final='gyr', mode=self.mode, galaxy=self.mode) - jn.timeconversion(jout, final='gyr', mode=self.mode, galaxy=self.mode)
            dt *= 1e9 * 365 * 24 * 60 * 60 # Gyr to sec
            fats = np.zeros(3).astype(int)
            for igal in igals:
                ivel = jn.rms(igal['vx'], igal['vy'], igal['vz'])
                isnap = self.data.load_snap(jout, prefix=prefix)
                radii = 5*max(igal['r'],1e-4) + 5*dt*ivel*isnap.unit['km']
                neighbors = jn.cut_sphere(agals, igal['x'], igal['y'], igal['z'], radii, both_sphere=True)
                # self.debugger.info(f"igal[{igal['id']}] len={len(neighbors)} in radii")
                
                if len(neighbors)>0:
                    neighbors = neighbors[neighbors['m'] >= igal['m']*masscut_percent/100]
                    # self.debugger.info(f"igal[{igal['id']}] len={len(neighbors)} after masscut {masscut_percent} percent")
                    if len(neighbors) > nfat:
                        rate = np.zeros(len(neighbors))
                        gals, gmpids = self.data.load_gal(jout, neighbors['id'], return_part=True, prefix=prefix, fromtm=False)
                        iout, istep = jn.ioutistep(igal, galaxy=self.galaxy, mode=self.mode)
                        _, checkpart = self.data.load_gal(iout, igal['id'], return_part=True, prefix=prefix, fromtm=False)
                        ith = 0
                        for gal, gmpid in zip(gals, gmpids):
                            if jn.atleast_isin(checkpart, gmpid):
                                ind = jn.large_isin(checkpart, gmpid)
                                rate[ith] = jn.howmany(ind, True)/len(checkpart)
                            ith += 1
                        ind = rate>0

                        # if not True in ind:
                        #     self.debugger.info("zero?! draw figure!")#
                        #     # neighbors = jn.cut_sphere(gals, igal['x'], igal['y'], igal['z'], 3*radii, both_sphere=True)
                        #     fig, ax = plt.subplots(figsize=(6,6))#
                        #     ax.set_facecolor("black")#
                        #     ax.set_xlim(igal['x']-radii, igal['x']+radii)#
                        #     ax.set_ylim(igal['y']-radii, igal['y']+radii)#

                        #     istars = self.data.load_star(iout, igal['id'], prefix=prefix, silent=False)
                        #     ax.scatter(istars['x'], istars['y'], s=1, alpha=0.2, color='white')
                        #     cir = plt.Circle((igal['x'], igal['y']), igal['r'], edgecolor='cyan', facecolor='none', alpha=0.7)#
                        #     ax.add_artist(cir)#
                        #     _, agemin, agemax, _ = jn.minmax(istars['age', 'Gyr'])
                        #     self.debugger.info(f"min_age={agemin}, max_age={agemax}")
                        #     ith = 0
                        #     for gal, gmpid in zip(gals, gmpids):
                        #         jstars = self.data.load_star(jout, gal['id'], prefix=prefix, silent=False)
                        #         sc = ax.scatter(jstars['x'], jstars['y'], s=0.5, alpha=0.1)#
                        #         cir = plt.Circle((gal['x'], gal['y']), gal['r'], edgecolor=sc.get_facecolors()[0], facecolor='none', alpha=0.6)#
                        #         ax.add_artist(cir)#
                        #         props = dict(boxstyle='round', facecolor='black', alpha=0.5)
                        #         ax.text(gal['x'], gal['y']+0.5*gal['r'], f"{gal['id']}", color=sc.get_facecolors()[0], ha='center', va='center', alpha=1, bbox=props)
                        #         ind = jn.large_isin(istars['id'], jstars['id'])
                        #         rate[ith] = jn.howmany(ind, True)/len(istars['id'])
                        #         self.debugger.info(f"[Common part check] ({igal['id']} at {igal['timestep']}) <-({rate[ith]*100:.2f} %)-> ({gal['id']} at {gal['timestep']})")
                        #         ith += 1
                        #     ind = rate>0
                        #     fig.savefig(f"root{self.branch.root['id']}_iout{jout:03d}_candidate{igal['id']}.png", dpi=200)
                        #     plt.close()

                        neighbors, rate = neighbors[ind], rate[ind]
                        # self.debugger.info(f"igal[{igal['id']}] len={len(neighbors)} after crossmatch")
                        if len(neighbors) > 0:
                            if len(neighbors) > nfat:
                                arg = np.argsort(rate)
                                neighbors = neighbors[arg][-nfat:]
                                # self.debugger.info(f"igal[{igal['id']}] len={len(neighbors)} after score sorting")
                            fats = np.concatenate((fats, neighbors['id']))
                    elif len(neighbors) > 0:
                        fats = np.concatenate((fats, neighbors['id']))
                    else:
                        pass
                
            fats = np.concatenate((fats, np.zeros(3).astype(int)))
        
        clock.done()
        return np.unique( fats[fats>0] ), jout
    

    def find_candidates(self, masscut_percent=1, nstep=5, nfat=5, prefix="", trust_tree=False, **kwargs):
        ############################################################
        ########    ADD case when no father, jump beyond     ####### Maybe done?
        ############################################################
        if not self.havefats:
            func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
            clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

            igals = np.atleast_1d(self.gal_tm)
            njump=0
            for i in range(nstep):
                fatids, jout = self.load_fatids(igals, njump=njump, masscut_percent=masscut_percent, nfat=nfat, prefix=prefix, trust_tree=trust_tree, **kwargs)
                iout = jn.step2out(self.istep-1-i, galaxy=self.galaxy, mode=self.mode)
                self.debugger.info(f"*** iout(istep)={iout}({self.istep-1-i}), fats={fatids}")
                if len(fatids) == 0:
                    self.debugger.info("JUMP!!")
                    # self.debugger.info("Check TreeMaker!")
                    # for igal in igals:
                    #     inst = self.branch.gm2tm(igal, prefix=prefix)
                    #     self.debugger.info(f"fat1:{inst['fat1']}, fat2:{inst['fat2']}, fat3:{inst['fat3']}, fat4:{inst['fat4']}, fat5:{inst['fat5']}")
                    njump += 1
                    if not self.havefats:
                        self.havefats=False
                else:
                    njump = 0
                    self.havefats=True
                    self.branch.update_cands(self.istep-1-i, fatids, checkids=self.part['id'], prefix=prefix) # -> update self.branch.candidates & self.branch.scores
                    igals = self.data.load_gal(self.istep-i-1, fatids, return_part=False, prefix=prefix, fromtm=True)

            clock.done()
        return self.havefats
    

    def calc_score(self, prefix=""):
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

        timekeys = self.branch.candidates.keys()
        for tout in timekeys:
            checkpart = self.part
            if self.galaxy:
                tgyr = jn.timeconversion(tout,start='nout', final='gyr', mode=self.mode, galaxy=self.galaxy)
                igyr = jn.timeconversion(self.iout, final='gyr', mode=self.mode, galaxy=self.galaxy)
                candidates = self.branch.candidates[tout]
                checkpart = self.part[ self.part['age','Gyr'] >= (igyr-tgyr) ]
            for i, ileaf in candidates.items():
                try:
                    score1 = self.calc_matchrate(ileaf, checkpart=checkpart, prefix=prefix) # importance weighted matchrate
                    score2 = 0  # Velocity offset
                    score3 = 0
                    if score1 > 0:
                        score2 = self.calc_velocity_offset(ileaf, prefix=prefix)
                        score3 = np.exp( -np.abs(np.log10(self.gal_gm['m']/ileaf.gal_gm['m'])) )   # Mass difference
                except Warning as e:
                    print("WARNING!! in calc_score")
                    breakpoint()
                    self.debugger.warning("########## WARNING #########")
                    self.debugger.warning(e)
                    self.debugger.warning(f"len parts {len(checkpart['m'])}")
                    self.debugger.warning(f"dtype {checkpart.dtype}")
                    self.debugger.warning(self.summary())
                    raise ValueError("velocity wrong!")
                self.branch.scores[tout][i] += score1+score2+score3
                self.branch.score1s[tout][i] += score1
                self.branch.score2s[tout][i] += score2
                self.branch.score3s[tout][i] += score3

        clock.done()


    def calc_matchrate(self, otherleaf, checkpart=None, prefix=""):
        # Subject to `calc_score`
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        # clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

        otherleaf.refer += 1
        if checkpart is None:
            checkpart = self.part
            if self.galaxy:
                tout = otherleaf.iout
                tgyr = jn.timeconversion(otherleaf.iout, final='gyr', mode=otherleaf.mode, galaxy=otherleaf.galaxy)
                igyr = jn.timeconversion(self.iout, final='gyr', mode=self.mode, galaxy=self.galaxy)
                checkpart = self.part[ self.part['age','Gyr'] >= (igyr-tgyr) ]
        if jn.atleast_isin(checkpart['id'], otherleaf.part['id']):
            ind = jn.large_isin(checkpart['id'], otherleaf.part['id'])
        else:
            return -1
        # clock.done()
        return np.sum( checkpart['m'][ind]/checkpart['dist'][ind] ) / np.sum( checkpart['m']/checkpart['dist'] )


    def calc_bulkmotion(self, checkpart=None, prefix="", **kwargs):
        # Subject to `calc_velocity_offset`
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        # clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

        if checkpart is None:
            checkpart = self.part

        try:
            vx = np.average(checkpart['vx', 'km/s'], weights=checkpart['m']/checkpart['dist'])
            vy = np.average(checkpart['vy', 'km/s'], weights=checkpart['m']/checkpart['dist'])
            vz = np.average(checkpart['vz', 'km/s'], weights=checkpart['m']/checkpart['dist'])
        except Warning as e:
            print("WARNING!! in calc_bulkmotion")
            breakpoint()
            self.debugger.warning("########## WARNING #########")
            self.debugger.warning(e)
            self.debugger.warning(f"len parts {len(checkpart['m'])}")
            self.debugger.warning(f"dtype {checkpart.dtype}")
            self.debugger.warning(self.summary())
            raise ValueError("velocity wrong!")

        # clock.done()
        return np.array([vx, vy, vz])

    
    def calc_velocity_offset(self, otherleaf, prefix="", **kwargs):
        # Subject to `calc_score`
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        # clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

        ind = jn.large_isin(otherleaf.part['id'], self.part['id'])
        if jn.howmany(ind, True) < 3:
            return 0
        else:
            refv = self.calc_bulkmotion(prefix=prefix)
            instar = otherleaf.part[ind]
            inv = self.calc_bulkmotion(checkpart=instar, prefix=prefix) - refv
            totv = np.array([otherleaf.gal_gm['vx'], otherleaf.gal_gm['vy'], otherleaf.gal_gm['vz']]) - refv
            # self.debugger.debug(f"instar velocity: {inv}")
            # self.debugger.debug(f"totstar velocity: {totv}")
            # self.debugger.debug(f"|delta v| = {np.sqrt( np.sum((totv - inv)**2) )}")
            # self.debugger.debug(f"|v_in|+|v_tot| = {np.linalg.norm(inv)+np.linalg.norm(totv)}")


        # clock.done()
        return 1 - np.sqrt( np.sum((totv - inv)**2) )/(np.linalg.norm(inv)+np.linalg.norm(totv))

# =Class: Branch=

In [15]:
#########################################################
###############         Branch Class                #####
#########################################################
class Branch():
    def __init__(self, root, DataObj, galaxy=True, mode='hagn', verbose=2, prefix="", debugger=None,trust_tree=False, **kwargs):
        func = f"[__Branch__]"; prefix = f"{prefix}{func}"
        clock = timer(text=prefix, verbose=verbose, debugger=debugger)
        self.debugger=debugger
        self.verbose = verbose

        self.root = root
        self.galaxy = galaxy
        self.mode = mode
        self.rootout, self.rootstep = jn.ioutistep(self.root, galaxy=self.galaxy, mode=self.mode)
        self.trust_tree = trust_tree
        if self.galaxy:
            self.key='star'
        else:
            self.key='dm'
        if mode[0]=='h':
            self.repo = f'/storage4/Horizon_AGN/'
            self.rurmode = "hagn"
        elif mode[0]=='y':
            self.repo = f'/storage3/Clusters/{mode[1:]}'
            self.rurmode = "yzics"
        else:
            raise ValueError(f"{mode} is not supported!")

        self.data = DataObj
                
        self.candidates = {} # dictionary of Leafs
        self.scores = {}
        self.score1s = {}
        self.score2s = {}
        self.score3s = {}
        self.rootleaf = self.gal2leaf(self.root, prefix=prefix)
        self.leaves = {self.rootout: self.gm2tm(self.root, prefix=prefix)} # results
        self.leave_scores = {self.rootout: 1} # results
        self.secrecord = 0
        self.go = True

        clock.done()

    def selfsave(self, dir="./data/"):
        a = self.root['id']
        b = self.secrecord
        c = self.leaves
        d = self.leave_scores
        readme = "1) root galaxy, 2) total elapsed time, 3) tree branch results, 4) corresponding score based on matchrate(importance-weighted) & mass difference & velocity offset"
        fname = f"Branch_{self.mode}_{a:05d}.pickle"
        self.debugger.info(f"\n>>> GAL{a:05d} is saved as `{fname}`")
        self.debugger.info(f">>> Treeleng = {len(c.keys())} (elapsed {self.secrecord/60:.2f} min)\n")
        print(f"\n>>> GAL{a:05d} is saved as `{fname}`")
        print(f">>> Treeleng = {len(c.keys())} (elapsed {self.secrecord/60:.2f} min)\n")
        jn.pklsave((readme, self.root,b,c,d), dir+fname, overwrite=True)

    def summary(self, onlyreturn=False):
        text ="\n[Summary report]\n>>> Root:" + "\n\t"
        if not onlyreturn:
            print(">>> Root:")
        text += jn.printgal(self.root, mode=self.mode, onlyreturn=onlyreturn) + "\n"
        if not onlyreturn:
            print(">>> Current root:")
        text += ">>> Current root:" + "\n\t"
        text += jn.printgal(self.rootleaf.gal_tm, mode=self.mode, onlyreturn=onlyreturn) + "\n"
        if not onlyreturn:
            print(">>> Current branch:")
        text += ">>> Current branch:" + "\n\t"
        if not onlyreturn:
            print([f"{a}:{b['id']}" if a%5==4 else f"{a}:{b['id']}\n" for a,b in self.leaves.items()])
        text += "["
        for a,b in self.leaves.items():
            if a%5==4:
                text += f"At {a}: {b['id']} | \n\t"
            else:
                text += f"At {a}: {b['id']} | "
        text += "]\n"
        if not onlyreturn:
            print(">>> Candidates:")
        text += ">>> Candidates:" + "\n"
        for iout in self.candidates.keys():
            keys = [f"{ikey}({self.scores[iout][ikey]:.4f}={self.score1s[iout][ikey]:.2f}+{self.score2s[iout][ikey]:.2f}+{self.score3s[iout][ikey]:.2f})" for ikey in self.candidates[iout].keys()]
            if not onlyreturn:
                print(f"[{iout}]\t{keys}")
            text += f"[{iout}]\t{keys}" + "\n"
        if not onlyreturn:
            print(f">>> Current Memory: {psutil.Process().memory_info().rss / 2 ** 30:.4f} GB")
        text += f">>> Current Memory: {psutil.Process().memory_info().rss / 2 ** 30:.4f} GB\n"
        text += f">>> Elapsed time: {self.secrecord:.4f} sec\n"
        return text

    def execution(self, prefix="", treeleng=None, **kwargs):
        print(f"Log is written in {self.debugger.handlers[0].baseFilename}")
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

        self.go = True
        while self.go:
            self.go = self.do_onestep(prefix=prefix, **kwargs)
            if treeleng is not None:
                if len(self.leaves.keys()) >= treeleng:
                    self.go=False
        clock.done()


    def do_onestep(self, prefix="", **kwargs):
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

        ref = time.time()
        try:
            self.go = self.rootleaf.find_candidates(prefix=prefix, trust_tree=self.trust_tree, **kwargs)
            # dprint(f"*** go? {self.go}", self.debugger)
            if len(self.candidates.keys())>0:
                self.rootleaf.calc_score(prefix=prefix)
                self.choose_winner(prefix=prefix)
            else:
                self.go = False
            if not self.go:
                while len(self.candidates.keys())>0:
                    self.choose_winner(prefix=prefix)
            
            self.secrecord += time.time()-ref
            clock.done()
            self.debugger.info(f"{prefix}\n{self.summary(onlyreturn=True)}")
        except Warning as e:
            print("WARNING!! in do_onestep")
            breakpoint()
            self.debugger.warning("########## WARNING #########")
            self.debugger.warning(e)
            self.debugger.warning(self.summary())
            raise ValueError("velocity wrong!")
        return self.go
        # try:
        #     self.rootleaf.find_candidates(prefix=prefix, trust_tree=self.trust_tree, **kwargs)
        #     self.rootleaf.calc_score(prefix=prefix)
        #     self.choose_winner(prefix=prefix)
        # except Warning:
        #     self.debugger.warning("########## WARNING #########")
        breakpoint()
        #     self.debugger.warning(self.summary())

        

    def reset_branch(self, gal, prefix=""):
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

        Newleaf = self.gal2leaf(gal, prefix=prefix)
        keys = list( self.leaves.keys() )
        for key in keys:
            if key < Newleaf.iout:
                refmem = MB()
                self.leaves[key] = None
                del self.leaves[key]
                self.debugger.debug(f"* [Branch][Reset] remove gal at iout={key} ({refmem-MB():.2f} MB saved)")
        keys = list( self.candidates.keys() )
        for key in keys:
            if key < Newleaf.iout:
                ids = list( self.candidates[key].keys() )
                for iid in ids:
                    refmem = MB()
                    self.candidates[key][iid].clear()
                    del self.candidates[key][iid]
                    self.debugger.debug(f"* [Branch][Reset] remove {iid} leaf at iout={key} ({refmem-MB():.2f} MB saved)")
                del self.candidates[key]
        gc.collect()

        clock.done()

    def gal2leaf(self, gal, prefix=""):
        iout, istep = jn.ioutistep(gal, galaxy=self.galaxy, mode=self.mode)
        if not iout in self.candidates.keys():
            self.candidates[iout] = {}
            self.scores[iout] = {}
            self.score1s[iout] = {}
            self.score2s[iout] = {}
            self.score3s[iout] = {}
        if not gal['id'] in self.candidates[iout].keys():
            func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
            # clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)
            
            self.candidates[iout][gal['id']] = Leaf(
                gal, self, self.data,
                verbose=self.verbose-1, prefix=prefix, debugger=self.debugger)
            self.scores[iout][gal['id']] = 0
            self.score1s[iout][gal['id']] = 0
            self.score2s[iout][gal['id']] = 0
            self.score3s[iout][gal['id']] = 0

            # clock.done()
        return self.candidates[iout][gal['id']]


    def update_cands(self, istep, galids,checkids=None, prefix=""):
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

        if self.trust_tree:
            gals = self.data.load_gal(istep, galids, return_part=False, prefix=prefix, fromtm=True)
        else:
            iout = jn.step2out(istep, galaxy=self.galaxy, mode=self.mode)
            if checkids is not None:
                gals, gmpids = self.data.load_gal(iout, galids, return_part=True, prefix=prefix, fromtm=False)
                for gal, gmpid in zip(gals, gmpids):
                    # dprint(f"*** <update_cands> ID{gal['id']}: {gal['nparts']} vs {len(gmpid)}", self.debugger)
                    # dprint(f"*** <update_cands> common parts = {jn.howmany(jn.large_isin(gmpid, checkids), True)}", self.debugger)
                    if jn.atleast_isin(gmpid, checkids):
                        self.gal2leaf(gal, prefix=prefix)
                        # dprint(f"*** <update_cands> ID{gal['id']} Gal becomes candidate!", self.debugger)
                    else:
                        pass
            else:
                gals = self.data.load_gal(iout, galids, return_part=False, prefix=prefix, fromtm=False)
                for gal in gals:
                    # dprint(f"*** <update_cands> ID{gal['id']} Gal becomes candidate!", self.debugger)
                    self.gal2leaf(gal, prefix=prefix)

        clock.done()


    def gals_from_candidates(self, istep, prefix=""):
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

        iout = jn.step2out(istep, galaxy=self.galaxy, mode=self.mode)
        cands = self.candidates[iout]
        for key in cands.keys():
            cands[key].refer += 1
        gals = np.concatenate([cands[key].gal_tm for key in cands.keys()])

        clock.done()
        return gals
    
    
    def choose_winner(self, prefix=""):
        func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
        clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

        iout = np.max(list(self.candidates.keys()))
        winid=0; winscore=0
        for iid, iscore in self.scores[iout].items():
            if iscore > winscore:
                winid = iid
                winscore = iscore
        if winscore<=0:
            pass
        else:
            self.debugger.debug(f"** (winner in {iout}) go? {self.go}")
            self.debugger.debug(f"** [Choose_winner] candidates\n{self.candidates}")
            self.debugger.debug(f"** [Choose_winner] scores {self.scores[iout]}")
            self.debugger.debug(f"** [Choose_winner] winid={winid}, winscore={winscore}")
            self.debugger.debug(f"* [Choose_winner] root leaf {self.rootleaf.gal_gm['id']} at {self.rootleaf.gal_gm['timestep']}")
            self.rootleaf = self.candidates[iout][winid]
            self.debugger.debug(f"* [Choose_winner] --> {self.rootleaf.gal_gm['id']} at {self.rootleaf.gal_gm['timestep']}")
            self.leaves[iout] = self.candidates[iout][winid].gal_tm
            self.leave_scores[iout] = winscore
    
        ids = list( self.candidates[iout].keys() )
        for iid in ids:
            refmem = MB()
            if iid != winid:
                self.candidates[iout][iid].clear()
            del self.candidates[iout][iid]
            del self.scores[iout][iid]
            self.debugger.debug(f"* [Choose_winner] remove {iid} leaf&score at iout={iout} ({refmem-MB():.2f} MB saved)")
        del self.candidates[iout]
        del self.scores[iout]

        ikeys = list( self.candidates.keys() )
        for ikey in ikeys:
            jkeys = list( self.candidates[ikey].keys() )
            for jkey in jkeys:
                if ikey-iout > 2:
                    self.candidates[ikey][jkey].refer -= 1
                if self.candidates[ikey][jkey].refer <= 0 or self.scores[ikey][jkey] <= -1:
                    self.debugger.debug(f"ikey={ikey}, jkey={jkey}, refer={self.candidates[ikey][jkey].refer}, score={self.scores[ikey][jkey]}")
                    refmem = MB()
                    self.candidates[ikey][jkey].clear()
                    del self.candidates[ikey][jkey]
                    del self.scores[ikey][jkey]
                    self.debugger.debug(f"* [Choose_winner] remove non-referred {jkey} leaf&score at iout={ikey} ({refmem-MB():.2f} MB saved)")
            if len(self.candidates[ikey].keys())<1:
                del self.candidates[ikey]
                del self.scores[ikey]

        gc.collect()        
        clock.done()

        
    def gm2tm(self, gal, prefix="",**kwargs):
        if 'nparts' in gal.dtype.names:
            func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
            # clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)

            iout, istep = jn.ioutistep(gal, galaxy=self.galaxy, mode=self.mode)
            
            # clock.done()
            return self.data.load_gal(istep, gal['id'], return_part=False, prefix=prefix, fromtm=True)
        else:
            return gal

        
    def tm2gm(self, gal, prefix="", **kwargs):
        if 'nparts' in gal.dtype.names:
            return gal
        else:
            func = f"[{inspect.stack()[0][3]}]"; prefix = f"{prefix}{func}"
            # clock = timer(text=prefix, verbose=self.verbose, debugger=self.debugger)
    
            iout, istep = jn.ioutistep(gal, galaxy=self.galaxy, mode=self.mode)
    
            # clock.done()
            return self.data.load_gal(iout, gal['id'], return_part=False, prefix=prefix, fromtm=False)

# Select Target

In [16]:
#########################################################
###############         Targets                ##########
#########################################################
Data = None
gc.collect()
print(f"\n>>> Current Memory: {psutil.Process().memory_info().rss / 2 ** 30:.4f} GB")
mode = input("Mode=?")
prefix = f"[{mode}]"
dm_nout, dm_nstep, dm_zred, dm_aexp, dm_gyr = jn.pklload(f'/storage6/jeon/data/{mode}/dm_{mode}_nout_nstep_zred_aexp_gyr.pickle')
nout, nstep, zred, aexp, gyr = jn.pklload(f'/storage6/jeon/data/{mode}/{mode}_nout_nstep_zred_aexp_gyr.pickle')
if mode[0]=='y':
    repo = f'/storage3/Clusters/{mode[1:]}'
    rurmode = 'yzics'
elif mode[0] == 'h':
    repo = '/storage4/Horizon_AGN'
    rurmode = 'hagn'
else:
    raise ValueError(f"{mode} is not supported!")
print(f"{prefix} {nout[-1]} ~ {nout[0]}")


>>> Current Memory: 0.3108 GB
[y07206] 20 ~ 187


In [17]:
import tracemalloc
import linecache

In [18]:
def display_top(snapshot, key_type='lineno', limit=10):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        print("#%s: %s:%s: %.1f KiB"
              % (index, frame.filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

In [19]:
jn.memory_usage()
tracemalloc.start()


[debug] memory usage:  318.26562 MB


In [20]:
iout = np.max(nout)
uri.timer.verbose = 0
snap_now = uri.RamsesSnapshot(repo, iout, path_in_repo='snapshots', mode=rurmode )
snap_now.set_box([0.5, 0.5, 0.5], extent=[0.1, 0.1, 0.1])
jn.memory_usage()

[debug] memory usage:  318.26953 MB


318.26953125

TypeError: unsupported operand type(s) for ** or pow(): 'tuple' and 'int'

In [125]:
def rms(*args):
    inst = 0
    for arg in args:
        inst += arg**2
    return np.sqrt(inst)

def rms2(*args):
    temp = [arg**2 for arg in args]
    return np.sqrt(np.sum(temp))

args = (0,1,2,3, 4)

ref = time.time()
for i in range(1000):
    rms(*args)
print(time.time()-ref)


ref = time.time()
for i in range(1000):
    rms2(*args)
print(time.time()-ref)





0.006813526153564453
0.030108213424682617


In [137]:
type(True)

bool

In [139]:
def count(target, vals):
    '''
    How many vals in target
    
    Examples
    --------
    >>> target = [1,1,2,3,7,8,9,9]
    >>> vals = [1,3,5,7,9]
    >>> howmany(target, vals)
    6
    '''
    if isinstance(vals, Iterable):
        return np.count_nonzero(np.isin(target, vals))
    else:
        if isinstance(vals, bool) and vals:
            print(vals)
            return np.count_nonzero(target)
        else:
            return np.count_nonzero(target==vals)

a = np.random.rand(10000000)>0.5

ref = time.time()
print( jn.howmany(a,True) )
print(time.time()-ref)

ref = time.time()
print( count(a,True) )
print(time.time()-ref)




5001842
0.0196382999420166
True
5001842
0.0014314651489257812


In [93]:
snap_now.params

{'nstar': array([8256872], dtype=int32),
 'star': array([ True]),
 'ncpu': 768,
 'ndim': 3,
 'levelmin': 7,
 'levelmax': 19,
 'ngridmax': 600000,
 'nstep_coarse': 9463,
 'boxlen': 1.0,
 'time': 3.18753606114813e-07,
 'aexp': 1.00000031724476,
 'H0': 70.4000015258789,
 'omega_m': 0.272000014781952,
 'omega_l': 0.727999985218048,
 'omega_k': 0.0,
 'omega_b': 0.045499999076128,
 'unit_l': 8.75000286134093e+26,
 'unit_d': 2.5343798131376e-30,
 'unit_t': 4.37500268106649e+17,
 'unit_m': 1.6978381420103502e+51,
 'h': 0.7040000152587891,
 'z': -3.1724465932025936e-07,
 'boxsize': 200.00000628800905,
 'boxsize_physical': 284.0910019915886,
 'boxsize_comoving': 284.09091186523546,
 'ordering': 'hilbert',
 'age': 13.741313851027979,
 'lookback_time': 0.0}

In [21]:
snap_now.get_part()

Particle({'table': array([(0.49362276, 0.4897057 , 0.4897835 , 0.00715612,  0.00412252, -0.0068322 , 3.97072125e-07, 0., 0.,  4988002, 7,   3),
       (0.49889795, 0.49709484, 0.46063257, 0.0104914 ,  0.00479353, -0.00523367, 3.97072125e-07, 0., 0.,   176430, 7,   3),
       (0.46351736, 0.45919322, 0.47663678, 0.00352212,  0.0041933 , -0.00540663, 3.97072125e-07, 0., 0.,   178667, 7,   3),
       ...,
       (0.50027412, 0.47863067, 0.49867063, 0.00989405,  0.00226173, -0.00634094, 3.97072125e-07, 0., 0.,  4988514, 7, 766),
       (0.50020601, 0.45113386, 0.49961932, 0.01313137,  0.00091928, -0.00580229, 3.97072125e-07, 0., 0.,  4988753, 7, 766),
       (0.54627923, 0.49981454, 0.46120806, 0.01405439, -0.01235758,  0.00137774, 3.97072125e-07, 0., 0., 10190565, 7, 766)],
      dtype=[('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<f8'), ('m', '<f8'), ('epoch', '<f8'), ('metal', '<f8'), ('id', '<i4'), ('level', '<i4'), ('cpu', '<i4')]), 'snap': <rur.uri.

In [22]:
snapshot = tracemalloc.take_snapshot()
display_top(snapshot)

Top 10 lines
#1: /home/jeon/rur/rur/uri.py:1717: 49482.5 KiB
    array = np.zeros(nitem, descr)
#2: /home/jeon/rur/rur/uri.py:905: 151.2 KiB
    part = part[mask]
#3: /home/jeon/.conda/envs/jeonpy/lib/python3.10/site-packages/numpy/core/records.py:428: 117.4 KiB
    self = ndarray.__new__(subtype, shape, (record, descr), order=order)
#4: <__array_function__ internals>:180: 6.9 KiB
#5: <__array_function__ internals>:179: 6.4 KiB
#6: /home/jeon/rur/rur/uri.py:314: 6.1 KiB
    self.bound_key = np.array(bound_key)
#7: /home/jeon/.conda/envs/jeonpy/lib/python3.10/site-packages/traitlets/traitlets.py:659: 3.7 KiB
    obj._trait_values[self.name] = value
#8: /home/jeon/.conda/envs/jeonpy/lib/python3.10/site-packages/traitlets/traitlets.py:1342: 3.5 KiB
    self.notify_change = hold  # type:ignore[assignment]
#9: /home/jeon/.conda/envs/jeonpy/lib/python3.10/site-packages/traitlets/traitlets.py:1375: 3.3 KiB
    Bunch(
#10: /home/jeon/.conda/envs/jeonpy/lib/python3.10/site-packages/traitlets/tr

In [23]:

print(f"\n{prefix} target galaxies load..."); ref = time.time()
iout = np.max(nout)
uri.timer.verbose = 0
snap_now = uri.RamsesSnapshot(repo, iout, path_in_repo='snapshots', mode=rurmode )
gals_now = uhmi.HaloMaker.load(snap_now, galaxy=True)
jn.printtime(ref, f"{prefix} {len(gals_now)} gals load done")
snap_now.clear()
print(f"\n>>> Current Memory: {psutil.Process().memory_info().rss / 2 ** 30:.4f} GB")

targets = gals_now
# targets = gals_now[np.argmin(gals_now['m'])]
# targets = gals_now[11:15]
# targets = gals_now[248]
targets = np.atleast_1d(targets)
# for target in targets:
#     jn.printgal(target, mode=mode)


[y07206] target galaxies load...
[y07206] 248 gals load done (0.025 sec elapsed)

>>> Current Memory: 0.3789 GB


In [24]:
#########################################################
###############         Debugger                #########
#########################################################
# import warnings
# warnings.simplefilter('error')

debugger = None
# fname = f"./output_{mode}.log"
fname = f"./output_dev.log"
if os.path.isfile(fname):
    os.remove(fname)
debugger = logging.getLogger(f"YoungTree_dev")
debugger.handlers = []
debugger.setLevel(logging.INFO)
formatter = logging.Formatter(u'%(asctime)s [%(levelname)8s] %(message)s')
file_handler = logging.FileHandler(fname, mode='a')
file_handler.setFormatter(formatter)
debugger.addHandler(file_handler)
debugger.propagate = False
debugger.debug("Debug Start")
print(f"\n>>> Current Memory: {psutil.Process().memory_info().rss / 2 ** 30:.4f} GB")


>>> Current Memory: 0.3789 GB


In [25]:
#########################################################
###############         Tree Making                ######
#########################################################
flush_GB = 10 + 0.3*len(targets)
print(f"Allow {flush_GB:.2f} GB Memory")
MyTree = Treebase(simmode=mode, debugger=debugger, verbose=0, flush_GB=flush_GB, loadall=True)
print(f"\n\nSee {fname}\n\nRunning...\n")

Allow 84.40 GB Memory
Tree building...
Tree building done (0.08 sec)
Tree building...
Tree building done (0.15 sec)


See ./output_dev.log

Running...



In [40]:
nout

array([187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175,
       174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162,
       161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149,
       148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136,
       135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123,
       122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110,
       109, 108, 107, 106, 105, 104, 103, 102, 101, 100,  99,  98,  97,
        96,  95,  94,  93,  92,  91,  90,  89,  88,  87,  86,  85,  84,
        83,  82,  81,  80,  79,  78,  77,  76,  75,  74,  73,  72,  71,
        70,  69,  68,  67,  66,  65,  64,  63,  62,  61,  60,  59,  58,
        57,  56,  55,  54,  53,  52,  51,  50,  49,  48,  47,  46,  45,
        44,  43,  42,  41,  40,  39,  38,  37,  36,  35,  34,  33,  32,
        31,  30,  29,  28,  27,  26,  25,  24,  23,  22,  21,  20])

In [56]:
fnames = os.listdir("/storage3/Clusters/07206/halo")
key = 'tree_bricks'
test = [True if key in file else False for file in fnames]

In [64]:
def load_nout(repo, galaxy=True):
    if galaxy:
        path = f"{repo}/galaxy"
    else:
        path = f"{repo}/halo"

    fnames = np.array(os.listdir(path))
    ind = [True if "tree_bricks" in file else False for file in fnames]
    fnames = fnames[ind]
    fnames = -np.sort( -np.array([int(file[11:]) for file in fnames]) )
    return fnames
load_nout('/storage3/Clusters/07206/', galaxy=False)

array([187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175,
       174, 173, 172, 171, 170, 169, 168, 167, 166, 165, 164, 163, 162,
       161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149,
       148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136,
       135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123,
       122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110,
       109, 108, 107, 106, 105, 104, 103, 102, 101, 100,  99,  98,  97,
        96,  95,  94,  93,  92,  91,  90,  89,  88,  87,  86,  85,  84,
        83,  82,  81,  80,  79,  78,  77,  76,  75,  74,  73,  72,  71,
        70,  69,  68,  67,  66,  65,  64,  63,  62,  61,  60,  59,  58,
        57,  56,  55,  54,  53,  52,  51,  50,  49,  48,  47,  46,  45,
        44,  43,  42,  41,  40,  39,  38,  37,  36,  35,  34,  33,  32,
        31,  30,  29,  28,  27,  26,  25,  24,  23,  22,  21,  20,  19,
        18,  17,  16,  15,  14,  13,  12,  11,  10,   9,   8,   

In [51]:
print(fnames)

None


In [26]:
import objgraph

In [27]:
objgraph.show_most_common_types(limit = 10)

dict              79003
list              58632
function          57167
Token             53122
tuple             32217
Name              15745
cell              13433
weakref           11916
Operator          7945
getset_descriptor 7879


NameError: name 'graphviz' is not defined

In [31]:
objgraph.show_refs(MyTree, filename='sample-graph.png')

Graph written to /tmp/objgraph-g1cv4t2u.dot (29 nodes)
Image renderer (dot) not found, not doing anything else


In [ ]:
jn.STOP()

ValueError: STOP!

In [ ]:
#########################################################
###############         Execution                ########
#########################################################
MyTree.queue(iout, targets, galaxy=True)
print("\nDone\n")

# Old indices

In [156]:
a = 3
b = {1:a, 2:a*2, 3:4}
print(b)
del b[1]
print(b)
print(a)

{1: 3, 2: 6, 3: 4}
{2: 6, 3: 4}
3


### YZiCS - mode

In [ ]:
# modes = [\
#     'y04466','y05420','y05427','y07206',\
#     'y10002','y17891','y24954','y29172',\
#     'y29176','y35663','y36413','y36415',\
#     'y39990','y49096'\
#         ]

# for mode in modes:
#     dm_nout, dm_nstep, dm_zred, dm_aexp, dm_gyr = jn.pklload(f'/storage6/jeon/data/{mode}/dm_{mode}_nout_nstep_zred_aexp_gyr.pickle')
#     nout, nstep, zred, aexp, gyr = jn.pklload(f'/storage6/jeon/data/{mode}/{mode}_nout_nstep_zred_aexp_gyr.pickle')
    # print(f"[Mode {mode}]STAR nout: {nout[-1]} ~ {nout[0]}")
    # print(f"[Mode {mode}]STAR nstep: {nstep[-1]} ~ {nstep[0]}")
    # print(f"[Mode {mode}]DM nout: {dm_nout[-1]} ~ {dm_nout[0]}")
    # print(f"[Mode {mode}]DM nstep: {dm_nstep[-1]} ~ {dm_nstep[0]}")

# Almost: 
#       STAR:   nout(20~187), nstep(1~168)
#       DM:     nout(10~187), nstep(1~178)

# Weird:
#   04466
#       DM:     nout(20~187), nstep(11~178)
#   05420
#       DM:     nout(20~187), nstep(1~168)
#   05427
#       DM:     nout(10~187), nstep(4~181)
#   17891
#       STAR:   nout(10~187), nstep(1~178)
#   24954
#       DM:     nout(11~187), nstep(2~178)
#   29172
#       STAR:   nout(15~187), nstep(1~173)
#       DM:     nout(12~187), nstep(3~178)
#   29176
#       STAR:   nout(18~187), nstep(1~170)
#   49096
#       DM:     nout(12~187), nstep(3~178)

In [ ]:
# modes = ['y07206', 'y10002', 'y35663', 'y36413', 'y36415', 'y39990']
# ngal: [34, 91, 151, 127, 107, 601]
modes = ['y07206', 'y36415', 'y36413', 'y10002']#, 'y39990']

mode = modes[0]
prefix = f"[{mode}]"
dm_nout, dm_nstep, dm_zred, dm_aexp, dm_gyr = jn.pklload(f'/storage6/jeon/data/{mode}/dm_{mode}_nout_nstep_zred_aexp_gyr.pickle')
nout, nstep, zred, aexp, gyr = jn.pklload(f'/storage6/jeon/data/{mode}/{mode}_nout_nstep_zred_aexp_gyr.pickle')
repo = f'/storage3/Clusters/{mode[1:]}'
print(f"{prefix} {nout[-1]} ~ {nout[0]}")

In [ ]:
# TreeMaker
print(f"\n{prefix} TreeMaker load..."); ref = time.time()
tm = jn.pklload(f"/storage6/jeon/data/{mode}/tm/{mode}_TreeMaker.pickle")
galtm = jn.treemaker(galaxy=True, tm=tm, mode=mode)
tm = None
jn.printtime(ref, f"{prefix} TreeMaker load done")

In [ ]:
# GalaxyMaker
print(f"\n{prefix} GalaxyMaker load..."); ref = time.time()
mem=jn.memory_usage()
gmpids = {}
cumpartss = {}
galaxymakers = {}
for i, iout in enumerate(nout):
    snap = uri.RamsesSnapshot(repo, iout, path_in_repo='snapshots', mode='yzics')
    gm, gmpid = uhmi.HaloMaker.load(snap, galaxy=True, load_parts=True)
    cumparts = np.insert(np.cumsum(gm["nparts"]), 0, 0)
    # gmpid = [inst] + [gmpid[ cumparts[i]:cumparts[i+1] ] for i in range(len(gm))]
    inst = np.copy(gmpid)
    gmpid = [inst[ cumparts[i]:cumparts[i+1] ] for i in range(len(gm))]
    galaxymakers[iout] = gm
    gmpids[iout] = gmpid
    snap.clear()
dict_done = {"galaxymakers":galaxymakers, "gmpids":gmpids}
jn.printtime(ref, f"{prefix} GalaxyMaker load done")

In [ ]:
print(f"\n{prefix} target galaxies load..."); ref = time.time()
iout = 187
snap_now = uri.RamsesSnapshot(repo, iout, path_in_repo='snapshots', mode='yzics' )
gals_now = uhmi.HaloMaker.load(snap_now, galaxy=True)
gals_now = gals_now[gals_now['m'] > 1e10]
jn.printtime(ref, f"{prefix} {len(gals_now)} gals load done")

### START FROM THIS

In [ ]:
from copy import deepcopy
# Young Tree setting
print(f"\n{prefix} YoungTree setting..."); ref = time.time()
yt = youngtree(galtm, gals_now, lengstep=3, mode=mode, verbose=0, debug=True)
yt.load_galaxymaker(dict_done=deepcopy(dict_done))
gc.collect()
jn.printtime(ref, f"{prefix} YT setting done")

### load fats 개선완료, 다른거수정

In [ ]:
# Young Tree Run
print(f"\n{prefix} YoungTree Run start..."); ref = time.time()
yt.Build_Tree(maxngal=10, agecut=True, mass_weight=True)
jn.printtime(ref, f"{prefix} YT Done")

In [ ]:
raise ValueError("STOP!")

In [ ]:
# Save tree
print(f"\n{prefix} YoungTree Run start..."); ref = time.time()
for target in gals_now:
    branch = yt.return_branch(target)
    jn.pklsave(branch, f"youngtree/YT_{mode}_{target['id']:05d}_branch.pickle")
jn.printtime(ref, f"{prefix} YT save Done")

### Bookmark

In [ ]:
raise ValueError("STOP!")

### 4. Make Catalogue

### 5. Build Model

### 6. Train Model

### 7. Evaluation